In [ ]:
require(randomForest)

## Score board

| No.   | score   |
| :---: | :---:   |
| 1     | 0.75120 |
| 2     | 0.75598 |


First try is even worse than the lady first approach.

## To improve

* Below I ignored Embarked, because in training data, some Embark is NA
  but in test data, Embark is all available, so the factor they give are different, randomForest cannot work with it.
  * This has been fixed, I just ignored the training data 
* I ignored "Name"/"Cabin"/"Ticket" because they are more than 53 level, which randomForest can not handle.
* There are some training data with NA entry, I ignored in the training, might need to include them
* Try to figure out why some of the predictions are NA

In [ ]:
train <- read.csv("train.csv")
# train.nona <- na.omit(train)
# drops <- c("PassengerId", "Name", "Cabin", "Ticket", "Embarked")
drops <- c("PassengerId", "Name", "Cabin", "Ticket")
train <- transform(train, Survived = as.factor(Survived))
train <- train[ , !(names(train) %in% drops)]
train <- subset(train, Embarked != "")
train <- transform(train, Embarked = as.factor(as.character(Embarked)))
# train <- train[train$Embarked != "",]
summary(train)

In [ ]:
e <- train$Embarked
str(e)

In [ ]:
rf.titanic <- randomForest(Survived ~ ., data = train, na.action = na.omit)

In [ ]:
test <- read.csv("test.csv")
test <- transform(test, Survived = 0)
test[1, "Survived"] <- 1
test <- transform(test, Survived = as.factor(Survived))
test <- test[ , !(names(test) %in% drops)]
summary(test)

In [ ]:
pred <- predict(rf.titanic, test)
str(pred)

In [ ]:
result.titanic <- data.frame(PassengerId = 892:1309, Survived = as.numeric(pred) - 1)

In [ ]:
summary(result.titanic)
str(result.titanic)
head(result.titanic)

* We can see quite a bit prediction is NA, so assume they are all not survived
* might be improved here:
* Try to figure out why they are NA first

In [ ]:
sum(is.na(result.titanic$Survived))
result.titanic$Survived[is.na(result.titanic$Survived)] <- 0

In [ ]:
result.titanic

In [ ]:
write.csv(result.titanic, file = "my_random_forest_2.csv", row.names = FALSE)

### The following part is trying to find out why some prediction